In [1]:
using DifferentialEquations
using Plots
using Interact
using Statistics

#using Pkg
#Pkg.add("WebIO")
#using WebIO
#WebIO.install_jupyter_nbextension()



WebIO._IJuliaInit()

In [2]:
include("newaux.jl")

h!

In [3]:
function aux_bisection_search(f,target, lower_bd, upper_bd)
       if sum(abs.(lower_bd-upper_bd))<0.001
        return lower_bd
    else
        if( f((lower_bd+upper_bd)/2)>target)
            return aux_bisection_search(f,target,lower_bd,(lower_bd+upper_bd)/2)
        else
            return aux_bisection_search(f,target,(lower_bd+upper_bd)/2,upper_bd)
        end
    end 
end

function bisection_search(f,target, lower_bd, upper_bd)
    @assert f(upper_bd)>=target
    @assert f(lower_bd)<=target
    aux_bisection_search(f, target, lower_bd, upper_bd)
end

bisection_search (generic function with 1 method)

In [7]:
my_range = 0.0:0.01:2.0
βvec = [0.15;0.19]
cvec = [0.2;0.0]
@manipulate for σ =0.01:0.01:0.14, ω=0.001:0.001:0.01, υ=0.1:0.05:10, c_star=range(extrema(cvec)...,length=51), 
    ρ=my_range
    
    g0 = SIRS_Game(2,fp)

    g0.x_star
    g0.σ = σ
    g0.ω = ω
    g0.γ = g0.σ
    g0.υ = υ
    g0.β   = βvec
    g0.c   = cvec
    g0.c_star = c_star
    g0.ρ = ρ

    fixall!(g0)
    
    @manipulate for I_0 = range(Ib(g0,g0.β[1]),stop=Ib(g0,g0.β[end]),length=15)
        lower_bd_index = findlast(x->x<=I_0, [Ib(g0,i) for i in g0.β] )
        upper_bd_index = findfirst(x->x>=I_0, [Ib(g0,i) for i in g0.β] )
        lower_bd = zeros(size(g0.β))
        upper_bd = zeros(size(g0.β))
        lower_bd[lower_bd_index] = 1.0
        upper_bd[upper_bd_index] = 1.0
        x_at_t0 = bisection_search(x->Ib(g0,g0.β'*x), I_0, lower_bd, upper_bd)
        β_at_t0 = g0.β'*x_at_t0
        
        
        I = Ib(g0,β_at_t0)
        R = Rb(g0,β_at_t0)
        S = 1.0-I-R

        W = [g0.β[1]*I;g0.β[1]*R;x_at_t0]

        ## assertions


        # betas are in increasing order                  # σ < β[1]
                        # c vector is in decreasing order                 #c_star > g0.c[end]
        if(all(diff(g0.β).>0) && all(diff(g0.c).<0) && all(g0.σ.<g0.β) && g0.c_star>g0.c[end] && (g0.c_star+g0.c[end]<g0.c[1]))
            fixall!(g0)
            prob = ODEProblem(h!,W ,[0.0,2500.0],g0)

            sol = solve(prob, AutoTsit5(Rosenbrock23()), save_everystep=true, saveat=0.1)  

            X = mapslices(x->[x;1.0-sum(x)], sol[xi(g0,1:g0.NS-1),:], dims=1)

            i_star = (g0.η*(1-g0.σ/g0.β_star))
            p1 = plot()
            Plots.plot!(p1, sol.t, (sol[1,:]./(g0.β'*X)')./(i_star), label="I,υ=$(g0.υ)")
            ylabel!(p1, "I/I*")
            xlabel!(p1, "Days")

            X = mapslices(x->[x;1.0-sum(x)], sol[xi(g0,1:g0.NS-1),:], dims=1)
            rr = (sol[qi(g0,1),:]'.*g0.β).+g0.r_star
            p2 = plot() 
            plot!(p2, sol.t, sum(X.*rr,dims=1)', label="cost(t),υ=$(g0.υ)")
            plot!(p2, x->g0.c_star,c=:black,linestyle=:dash, label=nothing)
            ylabel!(p2, "Cost - r(t)")
            xlabel!(p2, "Days")

            p3 = plot()
            plot!(p3, sol, vars=(3), label="x_1(t),υ=$(g0.υ)")
            ylabel!(p3, "x_1(t)")
            xlabel!(p3, "Days")


            t1 = plot(grid=false, axis=false, ticks=false)
            annotate!(0.05, 0.0, "Final", :left) 
            annotate!(0.05, 0.2, "x* = $(round.(g0.x_star;digits=2))", :left) 
            annotate!(0.05, 0.4, "β* = $(round(g0.β_star;digits=4))", :left) 
            annotate!(0.05, 0.6, "I* = $(round(g0.η*(1-g0.σ/g0.β_star);digits=4))", :left) 
            annotate!(0.05, 0.8, "R* = $(round((1-g0.η)*(1-g0.σ/g0.β_star);digits=4))", :left)
            
            
            annotate!(0.52, 0.0, "Initial", :left) 
            annotate!(0.52, 0.2, "x∘ = $(round.(x_at_t0;digits=2))", :left) 
            annotate!(0.52, 0.4, "β∘ = $(round(β_at_t0;digits=4))", :left) 
            annotate!(0.52, 0.6, "I∘ = $(round(I;digits=4))", :left) 
            annotate!(0.52, 0.8, "R∘ = $(round(R;digits=4))", :left)
            
            t1 = plot!()


            l = @layout [a t; b  c]
            plot(p1, t1, p2, p3, layout = l, size=(900,400))
        else
            #display error
            plot(grid=false, axis=false, ticks=false)
            
            annotate!(0.05, 0.9, "Error, all conditions bellow should be true:", :left)
            annotate!(0.05, 0.75, "all(diff(g0.β).>0) = $(all(diff(g0.β).>0))", :left) 
            annotate!(0.05, 0.6, "all(diff(g0.c).<0) = $(all(diff(g0.c).<0))", :left) 
            annotate!(0.05, 0.45, "all(g0.σ.<g0.β) = $(all(g0.σ.<g0.β))", :left) 
            annotate!(0.05, 0.3, "g0.c_star>0 = $(g0.c_star>0)", :left) 
            annotate!(0.05, 0.15, "(g0.c_star+g0.c[end]<g0.c[1]) = $(g0.c_star+g0.c[end]<g0.c[1])", :left)
            plot!(size=(900,400))


        end
    end
end  


Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["σ"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 14, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Any} with 2 listeners. Value:
7, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/jair/.julia/packages/Knockout/3gjS1/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/jair/.julia/packages/Knockout/3gjS1/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/jair/.julia/packages/InteractBase/h8Fwe/src/../assets/all.js"), Asset("css", nothing, "/home/jair/.julia/packages/InteractBase/h8Fwe/src/../assets/style.css"), Asset("css", nothing, "/home/jair/.julia/packages/Interact/pVyN2/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f45d973dba0, Task (runnable) @0x00007f45d973dba0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.01\",\"0.02\",\"0.03\",\"0.04\",\"0.05\",\"0.06\",\"0.07\",\"0.08\",\"0.09\",\"0.1\",\"0.11\",\"0.12\",\"0.13\",\"0.14\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"7047926735112067656\",\"id\":\"1813344761943182425\",\"type\":\"observable\"}),\"index\":WebIO.getval({